##Tensorflow based siamese network

In [0]:
import tensorflow as tf

In [2]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


###Siamese network definition

In [0]:
class siamese:

    # Create model
    def __init__(self):
        self.x1 = tf.placeholder(tf.float32, [None,4096])
        self.x2 = tf.placeholder(tf.float32, [None,4096])

        with tf.variable_scope("siamese") as scope:
            self.o1 = self.network(self.x1)
            scope.reuse_variables()
            self.o2 = self.network(self.x2)

        # Create loss
        self.y_ = tf.placeholder(tf.float32, None)
        
        self.loss = self.loss_with_spring()

    def network(self, x):
        weights = []
        fc1 = self.fc_layer(x, 1024, "fc1")
        ac1 = tf.nn.relu(fc1)        
        fc2 = self.fc_layer(ac1, 1024, "fc2")
        ac2 = tf.nn.relu(fc2)
        fc3 = self.fc_layer(ac2, 125, "fc3")
        
        return fc3

    def fc_layer(self, bottom, n_weight, name):
        assert len(bottom.get_shape()) == 2
        n_prev_weight = bottom.get_shape()[1]
        initer = tf.truncated_normal_initializer(stddev=0.01)
        W = tf.get_variable(name+'W', dtype=tf.float32, shape=[n_prev_weight, n_weight], initializer=initer)
        b = tf.get_variable(name+'b', dtype=tf.float32, initializer=tf.constant(0.01, shape=[n_weight], dtype=tf.float32))
        fc = tf.nn.bias_add(tf.matmul(bottom, W), b)
        return fc

    def loss_with_spring(self):
        margin = 5.0
        labels_t = self.y_
        labels_f = tf.subtract(1.0, self.y_, name="1-yi")          # labels_ = !labels;
        eucd2 = tf.pow(tf.subtract(self.o1, self.o2), 2)
        eucd2 = tf.reduce_sum(eucd2, 1)
        eucd = tf.sqrt(eucd2+1e-6, name="eucd")
        C = tf.constant(margin, name="C")
        # yi*||CNN(p1i)-CNN(p2i)||^2 + (1-yi)*max(0, C-||CNN(p1i)-CNN(p2i)||^2)
        pos = tf.multiply(labels_t, eucd2, name="yi_x_eucd2")
        # neg = tf.multiply(labels_f, tf.subtract(0.0,eucd2), name="yi_x_eucd2")
        # neg = tf.multiply(labels_f, tf.maximum(0.0, tf.subtract(C,eucd2)), name="Nyi_x_C-eucd_xx_2")
        neg = tf.multiply(labels_f, tf.pow(tf.maximum(tf.subtract(C, eucd), 0), 2), name="Nyi_x_C-eucd_xx_2")
        losses = tf.add(pos, neg, name="losses")
        loss = tf.reduce_mean(losses, name="loss")
        return loss

    def loss_with_step(self):
        margin = 3.0
        labels_t = self.y_
        labels_f = tf.subtract(1.0, self.y_, name="1-yi")          # labels_ = !labels;
        eucd2 = tf.pow(tf.subtract(self.o1, self.o2), 2)
        eucd2 = tf.reduce_sum(eucd2, 1)
        eucd = tf.sqrt(eucd2+1e-6, name="eucd")
        C = tf.constant(margin, name="C")
        pos = tf.multiply(labels_t, eucd, name="y_x_eucd")
        neg = tf.multiply(labels_f, tf.maximum(0.0, tf.subtract(C, eucd)), name="Ny_C-eucd")
        losses = tf.add(pos, neg, name="losses")
        loss = tf.reduce_mean(losses, name="loss")
        return loss

In [0]:
sess = tf.InteractiveSession()

In [5]:
# setup siamese network                                                                                                                                
sia = siamese();
train_step = tf.train.GradientDescentOptimizer(0.0005).minimize(sia.loss)
saver = tf.train.Saver()
tf.initialize_all_variables().run()

W0721 17:53:09.249661 140702633887616 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1205: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0721 17:53:09.414281 140702633887616 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf_should_use.py:193: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


###Performing data manipulation


In [0]:
image_train_data = np.load('/content/gdrive/My Drive/image features/xtrain.npy')
image_train_labels = np.load('/content/gdrive/My Drive/image features/ytrain.npy')

sketch_train_data = np.load('/content/gdrive/My Drive/feature_vectors_final/sketches_train_collated/x_train_sky.npy')
sketch_train_labels = np.load('/content/gdrive/My Drive/feature_vectors_final/sketches_train_collated/labels.npy')

In [0]:
image_train_labels = np.squeeze(image_train_labels)

In [0]:
image_train_labels

array([ 41., 114.,  95., ...,  12.,  59.,  58.])

In [0]:
image_train_data.shape

(58401, 4096)

In [0]:
sketch_train_data.shape

(58080, 4096)

In [0]:
args = np.argsort(sketch_train_labels)
argi = np.argsort(image_train_labels)

sorted_sklabel = sketch_train_labels[args]
sorted_imglabel = image_train_labels[argi]

sorted_sketch = sketch_train_data[args]
sorted_image = image_train_data[argi]

counti = 0
counts = 0

mini = None

spoint = 0
ipoint = 0

skdat = np.array([])
sklab = np.array([])

imgdat = np.array([])
imglab = np.array([])

sketch_train_data = 0
sketch_train_labels = 0
image_train_data = 0
image_train_labels = 0

for i in set(sorted_sklabel):
    print(i)
    counts = 0
    counti = 0
    for j in sorted_sklabel:
        if j == i:
            counts+=1
    
    for j in sorted_imglabel:
        if j == i:
            counti+=1
    
    if counts>counti:
        mini = counti
        
    else:
        mini = counts
        
    skdat = np.append(skdat,sorted_sketch[spoint:mini+spoint])
    sklab = np.append(sklab,sorted_sklabel[spoint:mini+spoint])
    imgdat = np.append(imgdat, sorted_image[ipoint:mini+ipoint])
    imglab = np.append(imglab,sorted_imglabel[ipoint:mini+ipoint])
    
    ipoint+=counti
    spoint+=counts      

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124


In [0]:
skdat = np.asarray(skdat)
sklab = np.asarray(sklab)
imgdat = np.asarray(imgdat)
imglab = np.asarray(imglab)

In [0]:
imglab.reshape(55831,1)

array([[  0.],
       [  0.],
       [  0.],
       ...,
       [124.],
       [124.],
       [124.]])

In [0]:
a = skdat
b = sklab
A = imgdat
B = imglab

In [0]:
skdat = 0
sklab = 0
imgdat = 0
imglab = 0

In [0]:
a = a.reshape((55831,-1))
A = A.reshape((55831, -1))

In [0]:
b = b.reshape((55831, -1))
B = B.reshape((55831, -1))

In [0]:
np.save('/content/gdrive/My Drive/feature_vectors_final/siamese/skdat.npy',a)
np.save('/content/gdrive/My Drive/feature_vectors_final/siamese/sklab.npy',b)
np.save('/content/gdrive/My Drive/feature_vectors_final/siamese/imgdat.npy',A)
np.save('/content/gdrive/My Drive/feature_vectors_final/siamese/imglab.npy',B)


###Training pairs


In [0]:
import numpy as np

sketchdata = np.load('/content/gdrive/My Drive/feature_vectors_final/siamese/skdat.npy')      #sketch feature data
sketchlabels = np.load('/content/gdrive/My Drive/feature_vectors_final/siamese/sklab.npy')    #sketch labels
imagedata = np.load('/content/gdrive/My Drive/feature_vectors_final/siamese/imgdat.npy')      #image feature data
imagelabels = np.load('/content/gdrive/My Drive/feature_vectors_final/siamese/imglab.npy')    #image labels


In [0]:
sketchlabels = np.squeeze(sketchlabels)
imagelabels = np.squeeze(imagelabels)

In [8]:
imagelabels.shape

(55831,)

In [0]:
image_index = []
for i in range(len(imagelabels)):
    image_index.append(i)
    
image_index = np.array(image_index)

import pandas as pd
image_dict = pd.DataFrame({'label':imagelabels, 'index':image_index})        #dataframe with label and index

In [12]:
sketch_index = []
image_index = []

label = []

upper = 10

for i in range(len(sketchlabels)):         #loop through all sketches
    
    if i %100 == 0:
        print(i)
        
    
    countpos = 0
    countneg = 0
    
    
    image_dict = image_dict.sample(frac=1).reset_index(drop=True)  #shuffling the images

    
    for j in range(len(image_dict['label'])):
        #loop through shuffled images
        
        
        if sketchlabels[i] == image_dict['label'][j] and countpos<upper:
            sketch_index.append(i)                         #store the index of sketch
            image_index.append(image_dict['index'][j])     #store the index of corresonding image
            label.append(1)
            countpos+=1
            
            
        else:
            if countneg<upper:
                sketch_index.append(i)
                image_index.append(image_dict['index'][j])
                label.append(0)
                countneg+=1
                
                
        
        if countpos ==upper and countneg==upper:
            break
            
            
        

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [0]:
import pandas as pd

dict = {'sketch index':sketch_index, 'image index':image_index, 'label':label}
df = pd.DataFrame(dict)      #making a dataframe of sketch, image and label indices
df.to_csv('/content/gdrive/My Drive/TrainingIndex.csv')

In [0]:
import pandas as pd
df = pd.DataFrame()
df = pd.read_csv('/content/gdrive/My Drive/TrainingIndex.csv')

In [10]:
saver.restore(sess, '/content/gdrive/My Drive/models/siamese/tensorflowmodelv6/model')

W0721 17:53:36.187247 140702633887616 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [11]:
epochs = 80

for epoch in range(epochs):       #loop through 20 epochs
    
    print('epoch: ', epoch+1)

    batch_size = 128

    n_batch = len(df['sketch index']) // batch_size    #number of batches
    
    df = df.sample(frac=1).reset_index(drop=True)  #shuffling the data i.e. indices

    
    loss = 0
    
    for i in range(n_batch):

                
        
        batch_x1 = np.array([sketchdata[x] for x in df['sketch index'][(i*batch_size):(i+1)*batch_size]])    #batch of sketches

        batch_x2 = np.array([imagedata[x] for x in df['image index'][(i*batch_size):(i+1)*batch_size]])     #batch of images

        batch_y = np.array([x for x in df['label'][(i*batch_size):(i+1)*batch_size]])         #batch of labels



        _, loss_v = sess.run([train_step, sia.loss], feed_dict={
                            sia.x1: batch_x1,
                            sia.x2: batch_x2,
                            sia.y_: batch_y})
        
        if i%100 == 0:
            print('\t#batch: ' + str(i) + '\t#loss: ' +  str(loss_v) )
        
        loss += loss_v
        

        if np.isnan(loss_v):
            print('Model diverged with loss = NaN')
            quit()
        
            
            
    print('loss: ', loss)
    saver.save(sess, '/content/gdrive/My Drive/models/siamese/tensorflowmodelv6/model')

    
                
            
            
        

epoch:  1
	#batch: 0	#loss: 2.5616553
	#batch: 100	#loss: 2.2965117
	#batch: 200	#loss: 2.9263835
	#batch: 300	#loss: 2.7421231
	#batch: 400	#loss: 2.5516367
	#batch: 500	#loss: 2.4864807
	#batch: 600	#loss: 2.8578978
	#batch: 700	#loss: 2.398942
	#batch: 800	#loss: 2.3499806
	#batch: 900	#loss: 2.9830084
	#batch: 1000	#loss: 2.4666743
	#batch: 1100	#loss: 2.7462025
	#batch: 1200	#loss: 2.8466358
	#batch: 1300	#loss: 2.4652808
	#batch: 1400	#loss: 2.8472354
	#batch: 1500	#loss: 2.7328558
	#batch: 1600	#loss: 2.5132294
	#batch: 1700	#loss: 2.8288696
	#batch: 1800	#loss: 2.919211
	#batch: 1900	#loss: 2.4109104
	#batch: 2000	#loss: 2.6440606
	#batch: 2100	#loss: 3.3355284
	#batch: 2200	#loss: 3.3895876
	#batch: 2300	#loss: 2.9094794
	#batch: 2400	#loss: 2.5740232
	#batch: 2500	#loss: 2.170853
	#batch: 2600	#loss: 2.5195913
	#batch: 2700	#loss: 2.9349399
	#batch: 2800	#loss: 2.5915124
	#batch: 2900	#loss: 2.4222348
	#batch: 3000	#loss: 3.0614245
	#batch: 3100	#loss: 2.7394059
	#batch: 3200

In [12]:
saver.save(sess, '/content/gdrive/My Drive/models/siamese/tensorflowmodelv6/model')

'/content/gdrive/My Drive/models/siamese/tensorflowmodelv6/model'

###MAP @ K


In [0]:
def eval_MAP_K(query_set, search_set, query_label, search_set_label, K):
    query_class_arr = query_set
    search_set_class_arr = search_set

    query_label_arr = query_label
    query_label_arr.astype(int)
    search_set_label_arr = search_set_label
    search_set_label_arr.astype(int)

    num_query_sample = query_label_arr.shape[0]

    dist_q_search_class = cdist(query_class_arr,search_set_class_arr,metric='euclidean')

    AP=0
    for query_count in range(0,num_query_sample):

        actual_label = query_label_arr[query_count]
        value=dist_q_search_class[query_count,:]
        sorted_value_idx=value.argsort()
        predicted_K_label = search_set_label_arr[sorted_value_idx[0:K]]
        L=np.sum((np.in1d(predicted_K_label,actual_label)).astype(int))

        P=0
        for r in range(0,K):
            rtr_label_r = predicted_K_label[:(r+1)]
            l=np.sum((np.in1d(rtr_label_r,actual_label)).astype(int))
            precision=l/(r+1)
            if actual_label==predicted_K_label[r]:
                delta=1
            else:
                delta=0
            P=P+(precision*delta)
        if L!=0:
            AP=AP+((1/L)*P)
    map_K=AP/num_query_sample
    return map_K

In [0]:
import numpy as np
sketch_set = np.load('/content/gdrive/My Drive/feature_vectors_final/sketches_test_collated/final.npy')
sketch_lab = np.load('/content/gdrive/My Drive/feature_vectors_final/sketches_test_collated/labels.npy')


In [0]:
image_set = np.load('/content/gdrive/My Drive/image features/xtest.npy')
image_lab = np.load('/content/gdrive/My Drive/image features/ytest.npy')

In [16]:
from sklearn.preprocessing import LabelEncoder

image_lab = LabelEncoder().fit_transform(image_lab)
sketch_lab = LabelEncoder().fit_transform(sketch_lab)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
saver.restore(sess, '/content/gdrive/My Drive/models/siamese/tensorflowmodelv6/model')

In [0]:
'''batch_size = 8
n_batches = len(image_set) // 8
image_representations = []
sketch_representations = []
for i in range(n_batches):
    img_repr = sess.run(sia.o1,{sia.o1: image_set[i*batch_size : (i+1)*batch_size]})
    image_representations.append(img_repr)

    
n_batches = len(sketch_set) // 8
   
for i in range(n_batches):
    sketch_repr = sess.run(sia.o1,{sia.o1: sketch_set[i*batch_size : (i+1)*batch_size]})
    sketch_representations.append(sketch_repr)
    '''

In [0]:
search_feat_sketch = sess.run(sia.o1,{sia.x1:sketch_set})
search_feat_image = sess.run(sia.o2,{sia.x2:image_set})

In [19]:
from scipy.spatial.distance import cdist
eval_MAP_K(search_feat_sketch, search_feat_image, sketch_lab, image_lab, 100)

0.22000519634688206

###Ambiguous Sketches

In [0]:
ambig_sketch_data = np.load('/content/gdrive/My Drive/feature_vectors_final/sketches_ambiguous_collated/final.npy')
ambig_sketch_label = np.load('/content/gdrive/My Drive/feature_vectors_final/sketches_ambiguous_collated/labels.npy')

In [21]:
ambig_search_feat_sketch = sess.run(sia.o1,{sia.x1:ambig_sketch_data})
search_feat_image = sess.run(sia.o2,{sia.x2:image_set})

from scipy.spatial.distance import cdist
eval_MAP_K(ambig_search_feat_sketch, search_feat_image, ambig_sketch_label, image_lab, 100)

0.011923973820010725